In [1]:
from nltk.tokenize import word_tokenize
import xml.etree.ElementTree as etree
from nltk.corpus import stopwords
from collections import Counter
import pandas as pd
import numpy as np
import sys
import re

In [2]:
sys.path.insert(0, '..')
from email_body_cleaner import clean_email_body

In [3]:
# Access the root of the XML document
root = etree.parse('../data/qit_unisi/HelpDeskRizzo2b.xml').getroot()

In [4]:
columns = [
    'id', 
    'subject', 
    'content', 
    'servizio',
    'ambito',
    'intervento'
]

# This will hold dataframe values
dataframe_values = []

# Extract relevant information from the XML file
for row in root.findall('records/row'):
    
    node_attrs = row.findall('column')
    node_attrs = list( map(lambda col: col.text, node_attrs) )
    node_attrs = np.array(node_attrs)
    
    dataframe_values.append(node_attrs)
    
# Create the dataframe    
dataframe_values = np.reshape(dataframe_values, (-1, len(columns)))
dataframe = pd.DataFrame(dataframe_values, columns=columns)

In [5]:
pd.options.display.max_rows = 3000
dataframe

,id,subject,content,servizio,ambito,intervento
0,12377,Richiesta nuova casella condivisa,L'utente RUGGIERO PASQUALE (C.F.: xxxxxx99x99x...,Posta Elettronica,Posta elettronica istituzionale (docenti e per...,Creazione/Modifica/Revoca di una casella condi...
1,12404,thunderbird malfunzionante,segnalo che da questa mattina thunderbird non ...,Posta Elettronica,Posta elettronica istituzionale (docenti e per...,Informazioni e supporto funzionale
2,12408,Non funzionamento mail istituzionale Pucci Tom...,"Spett. QIT,\n\nda stamani la mia mail istituzi...",Posta Elettronica,Posta elettronica istituzionale (docenti e per...,Informazioni e supporto funzionale
3,12405,Comunicazione indirizzo mail privato,Comunicazione indirizzo mail privato\n=20=20=2...,Gestione utenze,Gestione utenze,Supporto recupero password unisiPass
4,12409,None,"Buongiorno,\n\nci sno problemi con lo scaricam...",Posta Elettronica,Posta elettronica istituzionale (docenti e per...,Informazioni e supporto funzionale
5,12411,ANNULLO RICHIESTA,...LA CASELLA MAIL THUNDERBIRD =C3=A8 RIPARTIT...,None,None,None
6,12412,Cambio sede di lavoro,"Buongiorno,\ncomunico che da oggi la mia nuova...",None,None,None
7,12414,problema titulus,anche titulus ha problemi?\n\n\n Application ...,Gestione utenze,Gestione utenze,Informazioni e supporto funzionale
8,12413,Nono funzionamento posta elettronica,"Buongiorno,\nsegnalo che stamani non riesco ad...",Posta Elettronica,Posta elettronica istituzionale (docenti e per...,Informazioni e supporto funzionale
9,12418,Comunicazione indirizzo mail privato,Comunicazione indirizzo mail privato\n=20=20=2...,Gestione utenze,Gestione utenze,Supporto recupero password unisiPass


In [6]:
dataframe.isnull().sum()

id              0
subject        49
content         0
servizio      398
ambito        401
intervento    406
dtype: int64

In [7]:
# Remove rows having 'servizio' = None
dataframe = dataframe.dropna(subset=['servizio'])

In [8]:
dataframe.isnull().sum()

id             0
subject       38
content        0
servizio       0
ambito         3
intervento     8
dtype: int64

In [9]:
# Top-level class labels. More fine-grained labels can be found in 'ambito' and 'intervento'
dataframe['servizio'].value_counts()

Gestione utenze                      1013
Gestione Postazioni di Lavoro         385
Posta Elettronica                     351
Rete e Sistemi                        205
Fonia                                 136
Strumenti collaborativi                51
Abuse/Data breach                      25
Siti web                               24
Firma Elettronica  (no didattica)       8
Streaming/Videoconferenza               2
Name: servizio, dtype: int64

In [10]:
# Mid-level class labels
dataframe['ambito'].value_counts()

Gestione utenze                                                             1013
Posta elettronica istituzionale (docenti e pers. tecnico-amministrativo)     304
Rete                                                                         180
IMAC (Install-Move-Add-Change)                                               170
Software                                                                     162
Fonia Fissa                                                                   88
Virtualizzazione e Desktop Virtuali                                           50
Cartelle condivise                                                            44
Posta elettronica istituzionale (studenti)                                    41
Gestione Rubrica Telefonica                                                   37
Indirizzi e-mail                                                              24
Siti web ospitati                                                             24
Sistemi                     

In [11]:
# Low-level class labels
dataframe['intervento'].value_counts()

Informazioni e supporto funzionale                               633
Supporto recupero password unisiPass                             615
Richiesta/Modifica/Revoca indirizzi IP                            94
Creazione/Modifica/Revoca di una casella di posta personale       74
Creazione/Modifica/Revoca di una casella condivisa e alias        69
Installazione/Aggiornamento/Disinstallazione SW standard          60
Malfunzionamento HW Postazione di Lavoro                          57
Malfunzionamento casella                                          45
Richieste fonia IP                                                44
Aggiornamento/Assistenza/Manutenzione rubrica telefonica          37
Malfunzionamento Wi-Fi                                            33
Malfunzionamento                                                  31
Installazione/disinstallazione periferiche di rete                31
Malfunzionamento fonia IP                                         26
Problematiche funzionali          

As we can see, there are some duplicate rows (they have the same 'id'). The body of the email (column 'content') is the same for both, however in one case the body is wrapped in HTML tags. What we want to do is to remove such rows

In [12]:
# Add dummy column representing the number of characters of attribute 'content'
dataframe['content_length'] = list(map(len, dataframe['content'].values))

# Since it is evident that the row in which the email body is wrapped in HTML tags will have a larger value
# of 'content_length' compared to the one where the body is not wrapped in HTML tags, we do the following:
#  1) Sort rows by content_length in ascending order
#  2) Drop duplicate rows, keeping the first occurrence (namely the one whose content_length is minimum)
dataframe = dataframe.sort_values('content_length')
dataframe = dataframe.drop_duplicates('id', keep='first')

# Drop the dummy column introduced earlier on
dataframe = dataframe.drop(columns=['content_length'])

# Restore the original ordering of the dataframe's rows
dataframe = dataframe.sort_index()

/home/anfri/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
# Clean up emails bodies, as there are a lot of weird characters and newlines
dataframe['content'] = dataframe['content'].apply(clean_email_body)

# Remove rows having an empty 'content'
content_len = dataframe['content'].apply(len)
dataframe = dataframe.drop(index=content_len[content_len == 0].index.values)

In [14]:
# # Indices of examples containing english text, html tags and other things
# bad_rows_indices = [83, 93, 98, 99, 128, 129, 168, 250, 274, 
#                     295, 330, 367, 369, 375, 376, 381, 463, 
#                     490, 499, 557, 578, 597, 599, 600, 604, 
#                     618]

# dataframe = dataframe.drop(index=bad_rows_indices)

In [15]:
special_characters_regex_pattern = '[\W_]+'
single_letter_regex_pattern = '[a-zA-Z]'
numbers_regex_pattern = '[0-9]+'
hours_regex_pattern = '\d+[\.:]\d+([\.:]\d+)?'
dates_regex_pattern = '\d+[-/]\d+([-/]\d+)?'

regex = re.compile('^({spec}|{lett}|{num}|{hour}|{date})$'.format(spec=special_characters_regex_pattern, 
                                                                  lett=single_letter_regex_pattern, 
                                                                  num=numbers_regex_pattern,
                                                                  hour=hours_regex_pattern,
                                                                  date=dates_regex_pattern))

words = []
sequences = []
for email_body in dataframe['content'].values:

    tokens = word_tokenize(email_body.lower(), language='italian')
    
    # Remove tokens that match the regular expression
    tokens = [tok for tok in tokens if not regex.search(tok)]

    # Remove stopwords
    tokens = [tok for tok in tokens if not tok in stopwords.words('italian')]
    
    words.extend(tokens)
    sequences.append(tokens)

# words = list( map(str.lower, words) )    
vocabulary = list( set(words) )

In [16]:
Counter(words).most_common(1000)

[('siena', 3057),
 ('xxx', 2512),
 ('xxxxx', 2451),
 ('xxxxxx', 1600),
 ('mail', 1382),
 ('indirizzo', 1301),
 ('data', 1279),
 ('codice', 1153),
 ('università', 1125),
 ('pubblicato', 1046),
 ('studi', 1020),
 ('privato', 988),
 ('password', 983),
 ('richiesta', 840),
 ('grazie', 784),
 ('saluti', 732),
 ('ip', 692),
 ('online', 648),
 ('ateneo', 648),
 ('numero', 636),
 ('caso', 626),
 ('telefono', 614),
 ('fiscale', 603),
 ('n.', 603),
 ('ogni', 587),
 ("dell'università", 580),
 ('buongiorno', 573),
 ('+0000', 572),
 ("l'università", 570),
 ('pervenuta', 566),
 ('struttura', 550),
 ('informatico', 546),
 ('controllo', 546),
 ('regolamento', 534),
 ('cordiali', 531),
 ('rapporto', 531),
 ('comunicazione', 530),
 ('contenente', 527),
 ('sottoscritto', 526),
 ('disciplinare', 524),
 ('xxxxxx99x99x999x', 523),
 ('formale', 523),
 ('titolare', 522),
 ('recupero', 522),
 ("all'albo", 522),
 ('emanato', 521),
 ('d.r', 521),
 ('b.u', 521),
 ('n.114', 521),
 ('via', 520),
 ('accettare', 519)

## Labels mapping: QIT Unisi -> Ambrogio

In [17]:
mapping_qit_ambrogio = {
    'Gestione utenze':                   'Amministrativa',
    'Gestione Postazioni di Lavoro':     'Tecnica',
    'Posta Elettronica':                 'Tecnica',
    'Rete e Sistemi':                    'Tecnica',
    'Fonia':                             'Tecnica',
    'Strumenti collaborativi':           'Tecnica',
    'Abuse/Data breach':                 'Informazioni',
    'Siti web':                          'Tecnica',
    'Firma Elettronica  (no didattica)': 'Amministrativa',
    'Streaming/Videoconferenza':         'Informazioni'
}

In [18]:
dataframe['servizio'] = dataframe['servizio'].apply(mapping_qit_ambrogio.get)

In [19]:
dataframe['servizio'].value_counts()

Tecnica           1141
Amministrativa    1012
Informazioni        26
Name: servizio, dtype: int64

In [17]:
tickets = list( map(lambda seq: ' '.join(seq), sequences) )

np.savetxt('x_test_QIT.txt', tickets, fmt='%s', delimiter='\n')
np.savetxt('y_test_QIT.txt', dataframe['servizio'].values, fmt='%s', delimiter='\n')